In [1]:
import numpy as np, matplotlib.pyplot as plt, pandas as pd
pd.set_option('display.max_rows', 8)
!date

%load_ext autoreload
%autoreload 2

Wed Sep 18 18:29:34 PDT 2019


# Table of DALYs in total population, due to wasting

In [2]:
%cd /share/costeffectiveness/results/vivarium_conic_sam_comparison

/ihme/costeffectiveness/results/vivarium_conic_sam_comparison


In [3]:
import glob
# sorted(glob.glob('*/*/output.hdf'))

In [4]:
location_list = ['India', 'Bangladesh', 'Pakistan', 'Malawi', 'Tanzania', 'Mali']
fname = {}
for loc in location_list:
    fname[loc] = sorted(glob.glob(f'vivarium_conic_sam_comparison_{loc}/*/output.hdf'))[-1]
fname

{'India': 'vivarium_conic_sam_comparison_India/2019_09_12_14_36_19/output.hdf',
 'Bangladesh': 'vivarium_conic_sam_comparison_Bangladesh/2019_09_12_16_06_24/output.hdf',
 'Pakistan': 'vivarium_conic_sam_comparison_Pakistan/2019_09_12_14_36_44/output.hdf',
 'Malawi': 'vivarium_conic_sam_comparison_Malawi/2019_09_12_14_36_29/output.hdf',
 'Tanzania': 'vivarium_conic_sam_comparison_Tanzania/2019_09_12_14_36_50/output.hdf',
 'Mali': 'vivarium_conic_sam_comparison_Mali/2019_09_12_14_36_37/output.hdf'}

In [5]:
df = {}
for loc in location_list:
    df[loc] = pd.read_hdf(fname[loc])
    print(loc, len(df[loc]))

India 4000
Bangladesh 4000
Pakistan 4000
Malawi 4000
Tanzania 4000
Mali 4000


In [6]:
for loc in location_list:
    del df[loc]['random_seed']

In [7]:
scenarios = ['interventions.BEP_intervention.coverage_proportion',
 'interventions.SQ_LNS_intervention.coverage_proportion',
 'interventions.TF_SAM_intervention.coverage_proportion',]

for loc in location_list:
    g = df[loc].reset_index().groupby(scenarios + ['input_draw_number'])
    print(f'{loc} has {len(g.groups)} groups, with {len(df[loc])/len(g.groups):.2f} reps per group')
    df[loc] = g.sum()


India has 400 groups, with 10.00 reps per group
Bangladesh has 400 groups, with 10.00 reps per group
Pakistan has 400 groups, with 10.00 reps per group
Malawi has 400 groups, with 10.00 reps per group
Tanzania has 400 groups, with 10.00 reps per group
Mali has 400 groups, with 10.00 reps per group


In [8]:
baseline = (0,0,0)
bep = (.8, 0, 0)
sqlns = (0, .8, 0)
tfsam = (0, 0, .8)

In [9]:
sorted(df[loc].filter(like='time').columns)

['person_time_in_child_stunting_cat1',
 'person_time_in_child_stunting_cat2',
 'person_time_in_child_stunting_cat3',
 'person_time_in_child_stunting_cat4',
 'person_time_in_child_stunting_not_eligible',
 'simulation_run_time']

In [10]:
for loc in location_list:
    df[loc]['total_person_time'] = df[loc].filter(like='time').sum(axis=1)

In [11]:
scenario_map = dict(baseline=baseline, bep=bep, sqlns=sqlns, tfsam=tfsam)
cause_list = ('lower_respiratory_infections diarrheal_diseases measles protein_energy_malnutrition other_causes '
    + 'neonatal_preterm_birth neonatal_encephalopathy_due_to_birth_asphyxia_and_trauma hemolytic_disease_and_other_neonatal_jaundice neonatal_sepsis_and_other_neonatal_infections')\
        .split()

def dalys_per_100k(df, scenario, cat):
    
    scenario = scenario_map[scenario]
    
    # sam person years
    py = df.loc[scenario, 'total_person_time']
    dalys = pd.Series(0, index=py.index)
    for outcome in ['ylls', 'ylds']:
        for cause in cause_list:
            if outcome == 'ylds' and cause == 'other_causes':
                continue # YLDs for other causes is not tracked
            key = f'{outcome}_due_to_{cause}_in_child_stunting_{cat}'

            dalys += df.loc[scenario, key]
    
    return 100_000 * dalys / py
dalys_per_100k(df[loc], 'baseline', 'cat1').describe()

count      100.000000
mean      7234.432511
std       1074.087565
min       4414.778634
25%       6463.481466
50%       7191.665058
75%       7904.617703
max      10172.667328
dtype: float64

In [12]:
import pymc as pm

In [13]:
def my_formatted_output_pct(s):
    mu = f'{s.mean():.2f}'
    lb, ub = pm.utils.hpd(s, .05)
    lb = f'{lb:.2f}'
    ub = f'{ub:.2f}'
#     return [f'{mu} ({lb}, {ub})']
    return {'mean':mu, 'lb':lb, 'ub':ub}

In [15]:
def my_formatted_output_dalys(s):
    mu = f'{np.round(s.mean(), 1):.0f}'
    lb, ub = pm.utils.hpd(s, .05)
    lb = f'{np.round(lb, 1):.0f}'
    ub = f'{np.round(ub, 1):.0f}'
#     return [f'{mu} ({lb}, {ub})']
    return {'mean':mu}#, 'lb':lb, 'ub':ub}
my_formatted_output_dalys(dalys_per_100k(df[loc], 'baseline', 'cat2'))

{'mean': '13090'}

In [ ]:
results = {}

for cat in ['cat1', 'cat2']:
    for loc in location_list:
        val_0 = dalys_per_100k(df[loc], 'baseline', cat)
        for scenario in ['baseline', 'bep', 'sqlns', 'tfsam']:
            val_1 = dalys_per_100k(df[loc], scenario, cat)
            results[cat, loc, scenario, 'dalys'] = my_formatted_output_dalys(val_1)
            if scenario != 'baseline':
                results[cat, loc, scenario, 'dalys_averted'] = my_formatted_output_dalys(val_0-val_1)
                results[cat, loc, scenario, 'pct_averted'] = my_formatted_output_pct(100*(val_0-val_1)/val_0)
results = pd.DataFrame(results)        

In [ ]:
results = results.T.unstack().unstack().dropna(axis=1, how='all')

In [ ]:
# col_to_sort = (0, 'pct', 'baseline')
col_to_sort = ('mean', 'dalys', 'baseline')
results.loc['cat1'].sort_values(col_to_sort, ascending=False)

In [ ]:
results.loc['cat1'].loc[location_list, ('mean', 'dalys_averted')] # SAM DALYs per total person years

In [ ]:
results.loc['cat2'].loc[location_list, ('mean', 'dalys_averted')] # MAM DALYs per total person years